In [78]:
import requests
import chardet
from bs4 import BeautifulSoup
import facebook
import time
import re

cfg = {
 "page_id" : "328183278116459", # Step 1
 "access_token" : input('Access_token: ') #step3
}

graph = facebook.GraphAPI(cfg['access_token'])

###############Global setting end###############

def getrawlist():
    
    r = requests.get("http://bcc.hlis.hlc.edu.tw/files/40-1000-12-1.php")
    r.encoding = 'utf-8'
    #print(r.text)

    soup = BeautifulSoup(r.text,"html.parser")
    #print(soup.text)

    sel = soup.select("table.baseTB.listTB.list_TABLE.hasBD.hasTH tbody tr")
    #print(sel)
    
    ret = []

    for s in sel:
    #s = sel[0]
        #for begin
        data = {}
        
        #abstract title & url
        atag = s.select("a")
        data["title"] = atag[0]["title"].strip()
        data["url"] = atag[0]["href"].strip()
        
        #abstract issuer
        
        data["issuer"]  = s.select("td")[1].text.strip()
        
        #abstract date
        
        data["date"]  = s.select("td")[2].text.strip()
        
        ret.append(data)
        
        #for end
    
    return ret

def filt(dataset):
    
    #print(dataset)
    
    #get latest post on facebook page
    r_api = graph.get_object(id=cfg["page_id"],fields="posts.limit(1)")
    #print(r_api)
    
    #find the latest post title
    last_title = r_api['posts']['data'][0]['message'].strip()
    #print(last_title)
    
    #abstract title
    regex = re.compile(r'[^\/]*\/\s*(.*)\s+\/\s\d{4}\-\d{2}\-\d{2}') #match title
    match = regex.search(last_title)
    #print(type(match))
    
    if match is None:
        last_title = ""
    else:
        last_title = match[1]
    
    #print(last_title)
    
    #search title position
    for index,data in enumerate(dataset):
        if last_title != "" and last_title in data["title"]:
            #print("position:",index)

            #remove all datas after latest posted title(include it) *they're already posted
            for i in range(index,len(dataset)):
                dataset.pop()
                #print("delete:",dataset[i])
    
    #print(dataset)
    
    #make it ready for post
    ret = []
    for data in dataset:
        
        #print(data"\n")        
        
        ret.append({
            "message":'#'+data['issuer']+' / '+data['title']+' / '+data['date'],
            "link":data["url"]
        })
    
    return ret

def publisher(postset):
    
    r_api = []
    for post in postset:
        
        #post text,link
        r = graph.put_object(cfg['page_id'],"feed",message=post["message"],link=post["link"])
        #print(r)
        r_api.append(r)
        
        time.sleep(20)
        
    return r_api

#main

while True:
    
    now = time.localtime(time.time())
    
    print(now.tm_hour," : ")
    
    if now.tm_hour >= 6 and now.tm_hour <= 20:
        
        print("execute")
        
        rawlist = getrawlist()
        #print(rawlist)
        
        ready4post = filt(rawlist)
        #print(ready4post)
        
        result = publisher(ready4post)
        #print(result)
        
    else:
        
        print("sleep")
        time.sleep(3600)
    
    time.sleep(600) #run one per ten minutes





Access_token: EAAGK96361AABAKopkp7ll5Q4Ouvme7Cjm2wTMzE5wQTmW3SxoabfXnqtfXdsZBujROiUXgnZCriMVZBXjTE8Qqe7qKPtjXdiuTtAjIDtkpPDEs3ufFye34ztug3AlpDnGYotnYlwdBstscoRPAZASZAY153jHOXeylpdMOVJaAFdiO8aKQV0jDS8enN4dSwJ5TKdxayZC21AZDZD
<class 'NoneType'>
[{'message': '#輔導室 / 2019實踐家偏鄉青少年Money&You 654期公益營 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17379,r12-1.php'}, {'message': '#輔導室 / 重要訊息:通報不得超過24小時之規定 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17380,r12-1.php'}, {'message': '#教務處 / 公告107-2考程表及考試注意事項(本次考試備查證件為學生證、身分證、健保卡、駕照或勞動部所發證照擇一，具有清晰之大頭照、身分證字號、姓名者。) / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17378,r12-1.php'}, {'message': '#實習處 / 轉知由聖約翰科技大學舉辦「2020年第十五屆工業機器人競賽」 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17377,r12-1.php'}, {'message': '#學務處 / ~狂賀~本校跆拳隊陳倩、何中堯於2019世界中學生-技擊項目跆拳道錦標賽中勇奪一金、一銀!! / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17376,r12-1.php'}, {'message': '#總務處 / 108-01-02案公開標售本校

0

In [8]:
import requests
from bs4 import BeautifulSoup

r = requests.get("https://www.ptt.cc/bbs/MobileComm/index.html") #將網頁資料GET下來
soup = BeautifulSoup(r.text,"html.parser") #將網頁資料以html.parser
sel = soup.select("div.title a") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
for s in sel:
    #print(s["href"], s.text) 
    print(s)

<a href="/bbs/MobileComm/M.1560782814.A.EB9.html">[問題] 三星A60螢幕漏光</a>
<a href="/bbs/MobileComm/M.1560782950.A.3E6.html">[心得] 1+7 pro/ oneplus 7 pro7簡易開箱心得</a>
<a href="/bbs/MobileComm/M.1560783288.A.3D1.html">[問題] 華碩皇家俱樂部手機更換電池問題</a>
<a href="/bbs/MobileComm/M.1560783642.A.09F.html">[閒聊] 紅米K20 vs Realme X 拍攝比對</a>
<a href="/bbs/MobileComm/M.1560783757.A.290.html">[閒聊] ZenFone4 Android9更新心得</a>
<a href="/bbs/MobileComm/M.1560785833.A.685.html">[閒聊] XDA Reno 10X Zoom GCam使用討論</a>
<a href="/bbs/MobileComm/M.1534272593.A.93B.html">[情報] 3G要斷訊了? 3G執照到期相關情報整理</a>
<a href="/bbs/MobileComm/M.1546253879.A.29E.html">[公告] MobileComm 板規 181231</a>
<a href="/bbs/MobileComm/M.1546248617.A.0B0.html">[公告] 板規更新(討噓文、問機文、新聞格式、發推文格式)</a>
<a href="/bbs/MobileComm/M.1558366165.A.34A.html">[閒聊] 各種價位的手機/平板列表(2019/05/20)</a>
<a href="/bbs/MobileComm/M.1559572393.A.146.html">[閒聊] 置底閒聊文</a>


In [61]:
import facebook

cfg = {
 "page_id" : "328183278116459", # Step 1
 "access_token" : "EAAGK96361AABAM6hbO37HUsNeTZAb868ujDPNkU5qPIWrUBaqrisQYjjOsXk8t03QelbtiKMzHJP7LDLbIwUMyotnopuz8JKpnE31yWQ8s0jrzJ4yUX69OBdq38g2XMKq9tJbqBUAmMQRHWs5AKvuZAd3Yfw0QqHOH1UopbbthZBRo3EjjRjLOmUKel38NdpUnhLN57XwZDZD" #step3
}

graph = facebook.GraphAPI(cfg['access_token'])

r = graph.get_object(id=cfg["page_id"],fields="posts.limit(1)")

print("result:",r)

GraphAPIError: Error validating access token: Session has expired on Tuesday, 18-Jun-19 19:00:00 PDT. The current time is Tuesday, 18-Jun-19 21:10:58 PDT.

In [112]:
import time

now = time.localtime(time.time())
#print(now.tm_hour)
if now.tm_hour >= 6 and now.tm_hour <= 20:
            print("position:")
else:
    print("not")

position:
